In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
body = client_938e241641654cf0a8c1bb0aefbea33d.get_object(Bucket='default-donotdelete-pr-l16zufkrkht8vm',Key='df_train_sample.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_train_sample = pd.read_csv(body)
df_train_sample.head()


,TransactionID,isFraud,D13,D14,D12,id_04,id_03,D6,D9,D8,...,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
0,2987071.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,None,None,None,None,None,None,visa,debit,W,None
1,2988706.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.583333,5.583333,...,Found,T,T,T,Found,F,visa,credit,H,Found
2,2988937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,None,None,None,None,None,None,mastercard,debit,W,None
3,2988966.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,None,None,None,None,None,None,visa,debit,W,None
4,2988970.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,None,None,None,None,None,None,visa,credit,W,None


In [ ]:
df_valid

In [ ]:
# Keras model definition

In [10]:
# Sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
categorical_features = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
numeric_features = [c for c in df_train_sample.columns if c not in categorical_features and c not in ['TransactionID', 'isFraud']]


In [16]:
X_train = df_train_sample.drop(['TransactionID', 'isFraud'], axis=1)
y_train = df_train_sample.isFraud

In [ ]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', model(.......))])

clf.fit(X_train, y_train)